# 감성 분석 모델 학습 및 추론

### 1. 데이터 로드

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
df = pd.read_csv('train.tsv', sep='\t')
df

,comments,contain_gender_bias,bias,hate
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,False,others,hate
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,False,none,none
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",False,none,hate
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",False,none,none
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,True,gender,hate
...,...,...,...,...
7891,힘내세요~ 응원합니다!!,False,none,none
7892,힘내세요~~삼가 고인의 명복을 빕니다..,False,none,none
7893,힘내세용 ^^ 항상 응원합니닷 ^^ !,False,none,none
7894,힘내소...연기로 답해요.나도 53살 인데 이런일 저런일 다 있더라구요.인격을 믿습...,False,none,none


### 2. 데이터 전처리

In [9]:
# 1. 결측치 확인 및 처리
df.isna().sum()

comments               0
contain_gender_bias    0
bias                   0
hate                   0
dtype: int64

In [10]:
# label 생성: hate = 0(부정-악플), offensive = 1(부정), none = 2(긍정)으로 분류
labeling = { 'hate': 0, 'offensive': 1, 'none': 2 }

df['label'] = df['hate'].map(labeling)
df


,comments,contain_gender_bias,bias,hate,label
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,False,others,hate,0
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,False,none,none,2
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",False,none,hate,0
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",False,none,none,2
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,True,gender,hate,0
...,...,...,...,...,...
7891,힘내세요~ 응원합니다!!,False,none,none,2
7892,힘내세요~~삼가 고인의 명복을 빕니다..,False,none,none,2
7893,힘내세용 ^^ 항상 응원합니닷 ^^ !,False,none,none,2
7894,힘내소...연기로 답해요.나도 53살 인데 이런일 저런일 다 있더라구요.인격을 믿습...,False,none,none,2


In [ ]:
# 학습 데이터, 평가 데이터 분류
train_input, test_input, train_label, test_label = train_test_split(df['comments'], df['label'], test_size=0.25, random_state=42)

In [14]:
# TF-IDF 벡터화
tfidf_vectorizer = TfidfVectorizer()
train_input_vec = tfidf_vectorizer.fit_transform(train_input)
test_input_vec = tfidf_vectorizer.transform(test_input)

### 3. 모델 정의 및 생성

### 4. 모델 학습

In [57]:
# 모델 학습 진행
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(n_estimators=1000, learning_rate=0.25, random_state=42)
model.fit(train_input_vec, train_label)

,loss,'log_loss'
,learning_rate,0.25
,n_estimators,1000
,subsample,1.0
,criterion,'friedman_mse'
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_depth,3
,min_impurity_decrease,0.0
,init,None


### 5. 추론

In [59]:
# 추론에 사용할 문장
sentences = [
    "분필넣은 콧대..",
    "셀카 장인... 진짜 너무 예뻐!!!",
    "쓰레기 같은 놈.. ㅉㅉ",
    "T1 화이팅!!! 플옵 잘 준비해보자🍀",
    "나오면 누가 좋아한다고 자꾸 나옴? 보기 싫어",
    "악플 단 사람 면상은 어떤지..인성이 쓰레긴건 알겠고"
]

# 추론에 사용할 문장도 TF-IDF 벡터화
new_vec = tfidf_vectorizer.transform(sentences)

# 모델로 예측
pred_labels = model.predict(new_vec)

# 숫자 라벨 → 문자열 변환
label_mapping_rev = {0: '이건 선 넘었잖아🤬', 1: '마음씨를 곱게 쓰도록😠', 2: '긍정의 기운🥰'}
pred_labels_str = [label_mapping_rev[label] for label in pred_labels]

for sent, label in zip(sentences, pred_labels_str):
    print(f"문장: {sent} → 추론 : {label}")


문장: 분필넣은 콧대.. → 추론 : 마음씨를 곱게 쓰도록😠
문장: 셀카 장인... 진짜 너무 예뻐!!! → 추론 : 긍정의 기운🥰
문장: 쓰레기 같은 놈.. ㅉㅉ → 추론 : 이건 선 넘었잖아🤬
문장: T1 화이팅!!! 플옵 잘 준비해보자🍀 → 추론 : 긍정의 기운🥰
문장: 나오면 누가 좋아한다고 자꾸 나옴? 보기 싫어 → 추론 : 마음씨를 곱게 쓰도록😠
문장: 악플 단 사람 면상은 어떤지..인성이 쓰레긴건 알겠고 → 추론 : 이건 선 넘었잖아🤬
